<div align="right">  
    Grupo 13  
    
    André Neves da Costa - A95869 
    Filipe José Silva Castro - A96156 
</div>

# Variáveis de input

+ **N** $\rightarrow$ Número de estados que vai gerar a partir do estado inicial
+ **M** $\rightarrow$ Número de estados que vai gerar a partir do estado de erro

# Varáveis auxiliares

+ **a**, **b**, **c** e **d** $\rightarrow$ Variáveis que vão ter um bit gerado aleatoriamente e que vão definir o valor inicial dos inversores *A*, *B*, *D* e *C*, respetivamente

+ **caminhos** $\rightarrow$ Tuplo que contém 4 bits gerados aleatoriamente que vão definir que as transformações que vão ocorrer no bit de cada inversor, estando então, os valores das posições 0, 1, 2 e 3 do tuplo associados aos inversores *A*, *B*, *D* e *C*, respetivamente

+ **state** $\rightarrow$ Dicionário que contém o valor dos bits de todos os inversores em certo estado

# Implementação em python

A implementação começa com a atribuição de bits aleatórios às variáveis auxiliares, para isso usamos, da biblioteca random do python, o método *getrandbits*.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
from pysmt.typing import BOOL
import itertools
import random

a = random.getrandbits(1)
b = random.getrandbits(1)
c = random.getrandbits(1)
d = random.getrandbits(1)
caminhos = (random.getrandbits(1),random.getrandbits(1),random.getrandbits(1),random.getrandbits(1))

## `genState`

Recebe **s** e **i**:

+ **s** $\rightarrow$ String que definirá o nome do estado
+ **i** $\rightarrow$ Inteiro que definirá o número do estado

Recebe uma string e um inteiro e a partir dos quais cria um estado.

*pc1, pc2, pc3 e pc4 correpondem aos inversores A, B, D e C, respetivamente.*

In [2]:
def genState(s,i):
    state = {}
    state['pc1'] = Symbol("pc1"+'!'+ s +str(i),BVType(1))
    state['pc2'] = Symbol("pc2"+'!'+ s +str(i),BVType(1))
    state['pc3'] = Symbol("pc3"+'!'+ s +str(i),BVType(1))
    state['pc4'] = Symbol("pc4"+'!'+ s +str(i),BVType(1))
    return state

## `init`

Verifica se **state** é um estado inicial usando o seguinte predicado:

$$ (pc1 = a \wedge pc2 = b \wedge pc3 = c \wedge pc4 = d) $$

In [3]:
def init(state):
    return And(Equals(state['pc1'],BV(a,1)), Equals(state['pc2'], BV(b,1)),
        Equals(state['pc3'], BV(c,1)),Equals(state['pc4'], BV(d,1)))

## `error`

Verifica se **state** é o estado de erro usando o seguinte predicado:

$$ (pc1 = 0 \wedge pc2 = 0 \wedge pc3 = 0 \wedge pc4 = 0) $$

In [4]:
def error(state):
    return And(Equals(state['pc1'],BV(0,1)),Equals(state['pc2'],BV(0,1)),Equals(state['pc3'],BV(0,1)),Equals(state['pc4'],BV(0,1)))

## `trans`

Verifica que transformação é possível fazer entre os estados **curr** e **prox**.

Para isso verifica a variável **caminhos** para todos os inversores e caso tenha o valor 0, inverte o bit desse inversor, caso contrário, faz o Xor entre o bit desse inversor e o do intersor anterior.

In [5]:
def trans(curr,prox):
    if caminhos[0] == 0:
        t0 = And(Equals(prox['pc1'],BVNot(curr['pc1'])))
    else:
        t0 = And(Equals(prox['pc1'],BVXor(curr['pc1'], curr['pc4'])))
    if caminhos[1] == 0:
        t1 = And(Equals(prox['pc2'],BVNot(curr['pc2'])))
    else:
        t1 = And(Equals(prox['pc2'],BVXor(curr['pc2'],curr['pc1'])))
    if caminhos[2] == 0:
        t2 = And(Equals(prox['pc3'],BVNot(curr['pc3'])))
    else:
        t2 = And(Equals(prox['pc3'],BVXor(curr['pc3'], curr['pc2'])))
    if caminhos[3] == 0:
        t3 = And(Equals(prox['pc4'],BVNot(curr['pc4'])))
    else:
        t3 = And(Equals(prox['pc4'],BVXor(curr['pc4'],curr['pc3'])))
    
    return And(t0,t1,t2,t3)

## `bmc`

Aqui é implementado o algoritnmo de *bounded model checking* onde se dá um **n** e, através dos valores gerados aleatoriamente, o programa vai iterar por range(1,n+1) criando traços do estado inicial até ao estado n e, caso em alguma dessas iterações se encontre um erro, a execução para e devolve até que tamanho de traços é que dá para resolver o problema para os valores gerados.

In [6]:
def bmc(genState,init,trans,error,n):
    for k in range(1,n+1):
        with Solver(name="z3") as s:
            
            trace = [genState('X',i) for i in range(k)]
    
            s.add_assertion(init(trace[0]))
            for i in range(k-1):
                s.add_assertion(trans(trace[i],trace[i+1]))
                
            s.add_assertion(error(trace[-1]))
        
            if s.solve():
                for i in range(k):
                    print("Estado:",i)
                    for v in trace[i]:
                        print("          ",v,'=',s.get_value(trace[i][v]))
                break

    print(f"A propriedade é válida para traços de tamanho até {k-1}")
    


print(f"valores estado inicial: pc1 = {a}, pc2 = {b}, pc3 = {c}, pc4 = {d}")
print(f"caminhos: {caminhos}")                        
bmc(genState,init,trans,error,50)

valores estado inicial: pc1 = 1, pc2 = 1, pc3 = 1, pc4 = 1
caminhos: (1, 1, 1, 0)
Estado: 0
           pc1 = 1_1
           pc2 = 1_1
           pc3 = 1_1
           pc4 = 1_1
Estado: 1
           pc1 = 0_1
           pc2 = 0_1
           pc3 = 0_1
           pc4 = 0_1
A propriedade é válida para traços de tamanho até 1


## `invert`

Inverte as transições.


## `same`

Verifica se 2 estados são iguais.

## `baseName`

Pega em todos os caracteres antes de um "!" num string.


## `rename`

Substitui o nome das varáveis de um certo interpolante pelas presentes no estado dado.

In [8]:
def invert(trans):
    return (lambda u, v : trans(v,u))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

## `model_checking`

É usado o algoritmo *model checking* para verificar se o sistema é seguro.

Recebe **vars**, **init**, **trans**, **error**, **N**, **M**,

+ **vars** $\rightarrow$ Lista com os nomes das varáveis existentes em cada estado ['pc1','pc2','pc3','pc4']
+ **N** e **M** $\rightarrow$ Número de iterações que serão feitas pelo model checking a partir do estado inicial (*N*) e a partir do estado de erro (*M*)

Nesta implementação do algoritmo *model checking*, serão percorridos os estados dos inversores a partir do estado inicial e do estado de erro, verificando se em algum momento se intersetam, caso se intersetem é dado o sistema como inseguro, caso contrário  vai se verificar se o interpolante C é um invariante das transformações, se for, o sistema é seguro, senão, é procurado o majorante S, se for possível encontrar o majorante então o sistema é seguro, caso não seja possível, o ciclo volta ao início e é incrementado o valor de **n** ou **m**.

E é, então, feito isto até se atingir os valores máximo de **n** e **m** dados no input como **N** e **M**.

In [9]:
def model_checking(vars,init,trans,error,N,M):
    with Solver(name='z3') as s:
        
        X = [genState('X',i) for i in range(N+1)]
        Y = [genState('Y',i) for i in range(M+1)]
        
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1])
        
        for (n,m) in order:
             
            I = init(X[0])
            Tn = And([trans(X[i],X[i+1]) for i in range(n)])
            Rn = And(I,Tn)
            E = error(Y[0])
            Bm = And([invert(trans)(Y[i],Y[i+1]) for i in range(m)])
            Um = And(E,Bm)
            Vnm = And(Rn,same(X[n],Y[m]),Um)
            
            if s.solve([Vnm]):
                print(f"unsafe -> n:{n}, m:{m}")
            
            C = binary_interpolant(And(Rn,same(X[n],Y[m])), Um)
            
            if C is None:
                print("interpolant None")
                break
            
            C0 = rename(C,X[0])
            C1 = rename(C,X[1])
            T = trans(X[0],X[1])
            
            if not s.solve([C0,T,Not(C1)]):
                print(f"safe -> n:{n}, m:{m}")
                return 
            else:
                
                S = rename(C,X[n])
                while True:
                    A = And(S,trans(X[n],Y[m]))
                    if s.solve([A,Um]):
                        print("Não foi possível encontrar o majorante")
                        break
                    else:
                        Cnew = binary_interpolant(A,Um)
                        Cn = rename(Cnew,X[n])
                        
                        if s.solve([Cn,Not(S)]):
                            S = Or(S,Cn)
                        else:
                            print(f"safe -> n:{n}, m:{m}")
                            return 

print(f"valores estado inicial: pc1 = {a}, pc2 = {b}, pc3 = {c}, pc4 = {d}")
print(f"caminhos: {caminhos}")
model_checking(['pc1','pc2','pc3','pc4'],init,trans,error,4,4)

valores estado inicial: pc1 = 1, pc2 = 1, pc3 = 1, pc4 = 1
caminhos: (1, 1, 1, 0)
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante


# Testes

In [ ]:
--------------------------------------------------------------------|

valores estado inicial: pc1 = 1, pc2 = 1, pc3 = 1, pc4 = 1
caminhos: (0, 0, 0, 0)

------------------------ bmc   
    
Estado: 0
           pc1 = 1_1
           pc2 = 1_1
           pc3 = 1_1
           pc4 = 1_1
Estado: 1
           pc1 = 0_1
           pc2 = 0_1
           pc3 = 0_1
           pc4 = 0_1
A propriedade é válida para traços de tamanho até 1

------------------------ model_checking

Não foi possível encontrar o majorante
unsafe -> n:1, m:2
interpolant None

--------------------------------------------------------------------|

valores estado inicial: pc1 = 0, pc2 = 0, pc3 = 0, pc4 = 0
caminhos: (1, 1, 1, 1)
    
------------------------ bmc      
    
Estado: 0
           pc1 = 0_1
           pc2 = 0_1
           pc3 = 0_1
           pc4 = 0_1
A propriedade é válida para traços de tamanho até 0

------------------------ model_checking

unsafe -> n:1, m:1
interpolant None

--------------------------------------------------------------------|

valores estado inicial: pc1 = 0, pc2 = 1, pc3 = 1, pc4 = 1
caminhos: (0, 0, 0, 0)

------------------------ bmc     
    
A propriedade é válida para traços de tamanho até 49   | Não encontrou situação de erro em 50 iterações

------------------------ model_checking

Não foi possível encontrar o majorante   | Acontece infinitamente

--------------------------------------------------------------------|

valores estado inicial: pc1 = 0, pc2 = 0, pc3 = 1, pc4 = 0
caminhos: (1, 1, 1, 0)
    
------------------------ bmc     
    
Estado: 0
           pc1 = 0_1
           pc2 = 0_1
           pc3 = 1_1
           pc4 = 0_1
Estado: 1
           pc1 = 0_1
           pc2 = 0_1
           pc3 = 1_1
           pc4 = 1_1
Estado: 2
           pc1 = 1_1
           pc2 = 0_1
           pc3 = 1_1
           pc4 = 0_1
Estado: 3
           pc1 = 1_1
           pc2 = 1_1
           pc3 = 1_1
           pc4 = 1_1
Estado: 4
           pc1 = 0_1
           pc2 = 0_1
           pc3 = 0_1
           pc4 = 0_1
A propriedade é válida para traços de tamanho até 4

------------------------ model_checking

Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
unsafe -> n:1, m:3
interpolant None

--------------------------------------------------------------------|